In [1]:
import os

from tools import create_project_workbook
from autogen import ConversableAgent, GroupChat, GroupChatManager, register_function

In [2]:
# Define file path
file_path = "data/Client Script for POC (2024-06-26 10_34 GMT-3) - Transcript.txt"

with open(file_path, "r") as file:
    # Open the file and read the entire content into a single string
    file_content = file.read()

    # Print the word count and the content
    print(f"Word Count: {len(file_content)}")
    #print(file_content)

Word Count: 25661


In [3]:
# Agents' prompts
project_manager_agent_sys_msg = """You’re a seasoned project manager with over 15 years of experience in consulting clients on a diverse range of projects. You work for a company named Healthcare Facilitation Group (HFG) and specialize in developing comprehensive project workbooks that adhere to the PMBOK standards, ensuring that all elements of project management are effectively covered. Your knowledge extends across various industries, enabling you to tailor project management practices to meet unique client needs.
Please keep in mind the importance of aligning the project activities with the PMBOK standards and ensuring comprehensive coverage of all project management knowledge areas, including scope, time, cost, quality, human resources, communication, risk, procurement, and stakeholder management. It’s essential to produce a clear, well structured, and professional workbook that facilitates clear understanding and effective communication among all project team members."""

interviewing_agent_sys_msg = """You are a professional interviewer with extensive experience in conducting interviews with clients to gather essential information for the development of their project. Your expertise lies in asking insightful and open-ended questions that encourage clients to share detailed insights, ensuring that all key aspects of project management are addressed.
Your goal is to help the client articulate their vision and needs clearly. Aim to ensure that all essential project management aspects, including risks, resources, and expectations, are covered during the interview process."""

writing_agent_sys_msg = """You’re a professional project documentation specialist with over 15 years of experience in creating comprehensive project workbooks that align with PMBOK standards. Your expertise lies in meticulously detailing all aspects of project scope, schedule, resources, risks, and stakeholder engagement to ensure clarity and accountability throughout the project lifecycle.
Please ensure that the project workbook is structured according to PMBOK standards. Be mindful to maintain clarity and conciseness while ensuring that all necessary components are included for effective project tracking and management."""


In [4]:
# OpenAI configuration
# config_list = [
#     {
#         "model": "gpt-4",
#         "api_key": os.environ.get("OPENAI_API_KEY"),
#     }
# ]

# Local configuration
config_list = [
    {
        "model": "lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF",
        "api_type": "openai",
        "base_url": "http://localhost:1234/v1",
        "api_key": "NULL",
    }
]

# Create LLM configuration
llm_config = {
    #"seed": 42,
    "temperature": 0.7,
    "config_list": config_list,
}

In [5]:
# Define agents
project_manager_agent = ConversableAgent(
    name="project_manager",
    system_message=project_manager_agent_sys_msg,
    llm_config=llm_config,
    human_input_mode="NEVER",
)

interviewing_agent = ConversableAgent(
    name="interviewing_agent",
    system_message=interviewing_agent_sys_msg,
    llm_config=llm_config,
    human_input_mode="NEVER",
)

writing_agent = ConversableAgent(
    name="writing_agent",
    system_message=writing_agent_sys_msg,
    llm_config=False,
    human_input_mode="NEVER",
    default_auto_reply="No content to reply",
)

user_proxy = ConversableAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    max_consecutive_auto_reply=10,
    llm_config=False,
    human_input_mode="ALWAYS",
    code_execution_config={
        "work_dir": "agents-workspace",
        "use_docker": False,
    },
    default_auto_reply="No content to reply",
)

# Register the writing function to project_manager and writing_agent
register_function(
    create_project_workbook,
    caller=project_manager_agent,
    executor=writing_agent,
    name="writer",
    description="Used to create a project workbook document.",
)

# Create a group chat
group_chat = GroupChat(
    agents=[project_manager_agent, interviewing_agent, writing_agent, user_proxy],
    messages=[],
    max_round=5,
    send_introductions=True,
)

# Create the group chat manager
group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config,
    default_auto_reply="No content to reply",
)


In [6]:
# user_message = f"""
# Create a project workbook called “Workflow Improvement Project” from this client meeting.
# Write a short summary of what the project is about. Identify and describe the problem.
# Here is the transcript of the meeting with a client:
# {file_content}"""


# chat_result = user_proxy.initiate_chat(
#     recipient=group_chat_manager,
#     message=user_message,
#     summary_method="reflection_with_llm",
# )

In [7]:
user_message = f"""
Summarize this meeting:
{file_content}"""

user_message = f"""
What questions can I ask the client to gather essential information for the development of their project?
"""

user_message = f"""
Create a sample project workbook with random data and save it to a file.
"""

chat_result = user_proxy.initiate_chat(
    recipient=group_chat_manager,
    message=user_message,
    # summary_method="reflection_with_llm",
)

user_proxy (to chat_manager):


Create a sample project workbook with random data and save it to a file.


--------------------------------------------------------------------------------

Next speaker: writing_agent

writing_agent (to chat_manager):

No content to reply

--------------------------------------------------------------------------------

Next speaker: interviewing_agent

interviewing_agent (to chat_manager):

Here's an example of a sample project workbook based on the PMBOK standards:

**Project Workbook**

**Project Information**

* Project Name: Healthcare IT Implementation
* Project Sponsor: John Doe, CEO
* Project Manager: Jane Smith, HFG
* Start Date: 2023-02-01
* End Date: 2023-06-30

**Scope Management**

* Project Scope Statement:
	+ The project aims to implement a new Healthcare IT system to improve patient data management and streamline clinical workflows.
	+ The scope includes the following deliverables:
		- Design and development of the new system
		- Testing

BadRequestError: Error code: 400 - {'error': "'messages' array must only contain objects with a 'content' field that is not empty."}

In [ ]:
print(chat_result.summary)